In [57]:
import subprocess
import sys
import os

# Define a function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to install
packages = [
    "pandas",
    "numpy",
    "scikit-learn",
    "pickle-mixin",
    "pyarrow"
]

# Install each package
for package in packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

#assign correct directory
name = sys.platform

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC
import random as rm
from sklearn.metrics import classification_report, f1_score
import pickle as pk
from sklearn.preprocessing import PolynomialFeatures
from itertools import combinations
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Lasso
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import mahalanobis

In [58]:
import pandas as pd

# 1. Load the Stata dataset
df = pd.read_stata("CMF/CMF_1870.dta")


In [59]:
# count how many columns
print(df.columns)
# check if "hands_children" is in the columns
if "hands_children" in df.columns:
    print("hands_children is in the columns")
if "months_active" in df.columns:
    print("month_active is in the columns")

Index(['ind_detailed', 'ind_broadest', 'ind_leontief', 'ind_granular',
       'file_name', 'firm_number', 'industry_raw', 'industry_freq', 'fips',
       'state',
       ...
       'machine_category24', 'machine_unit25', 'machine_kind25',
       'machine_category25', 'machine_unit26', 'machine_kind26',
       'machine_category26', 'machine_unit27', 'machine_kind27',
       'machine_category27'],
      dtype='object', length=797)
hands_children is in the columns
month_active is in the columns


In [60]:
# 2. Convert "materials_value*" and "production_values*" columns to numeric (force conversion)
# Identify columns whose names start with either pattern and convert them (errors='coerce' converts non-numeric values to NaN)
for col in df.columns:
    if col == 'capital' or col == 'tot_wages' or col.startswith("hands") or col.startswith("materials_value") or col.startswith("production_values") or col == 'months_active':
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Generate aggregated features for "materials_value" and "production_values"
# For each base variable, sum all columns whose names start with that variable.
for base in ['materials_value', 'production_values']:
    matching_cols = [col for col in df.columns if col.startswith(base)]
    # Sum across the matching columns. The skipna=True means only non-missing numbers are summed.
    df[base] = df[matching_cols].sum(axis=1, skipna=True)
    # In case all values were missing, replace NaN with 0.
    df[base] = df[base].fillna(0)

# 4. Rename the aggregated 'production_values' column to 'output'
df.rename(columns={'production_values': 'output'}, inplace=True)

# 5. Keep only specific columns.
#    The Stata "keep" command retains:
#    file_name, firm_number, firm_name, output, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, all variables starting with "ind_",
#    materials_value1-5, and production_values1-5.
# First build a list of columns to keep:
base_keep = [
    'file_name', 'firm_number', 'firm_name', 'output', 'capital',
    'materials_value', 'hands_female', 'hands_male', 'hands_children', 'tot_wages', 'months_active']
# Add any columns starting with "ind_"
ind_cols = [col for col in df.columns if col.startswith("ind_")]
# Explicitly add materials_value1 through materials_value5 if they exist
mv_cols = [f"materials_value{i}" for i in range(1, 6) if f"materials_value{i}" in df.columns]
# Explicitly add production_values1 through production_values5 if they exist
# (These will later be renamed)
pv_cols = [f"production_values{i}" for i in range(1, 6) if f"production_values{i}" in df.columns]

# Retain only these columns (if they exist in the dataframe)
cols_to_keep = base_keep + ind_cols + mv_cols + pv_cols
df = df[cols_to_keep]

/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_3436/2395816782.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_3436/2395816782.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)


In [61]:
# 7. Rename production values variables
#    In Stata, the loop renames each variable starting with "production_values" (other than the aggregated one)
#    to output1, output2, etc.
# Find all columns that start with "production_values". (These are the individual ones kept earlier.)
pv_individual = [col for col in df.columns if col.startswith("production_values")]
for i, col in enumerate(pv_individual, start=1):
    df.rename(columns={col: f"output{i}"}, inplace=True)

In [62]:
# 8. Reorder columns so that a specific set of variables come first.
#    The desired order (moved to front) is:
#    file_name, firm_number, firm_name, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, output.
#    Other columns will follow in their original order.
front_cols = ['file_name', 'firm_number', 'firm_name', 'capital', 'hands_male',
              'hands_female', 'hands_children', 'tot_wages', 'months_active', 'materials_value', 'output']
remaining_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + remaining_cols]

# 9. Save the resulting dataframe to a Stata .dta file.
output_file = r"CMF_to_predict/1870_to_predict.dta"
df.to_stata(output_file, write_index=False)

print("Data processing complete. Output saved to:", output_file)

Data processing complete. Output saved to: CMF_to_predict/1870_to_predict.dta


In [63]:
# Read data
input_1870 = pd.read_stata("CMF_to_predict/1870_to_predict.dta")
# get the columns
input_1870.columns

Index(['file_name', 'firm_number', 'firm_name', 'capital', 'hands_male',
       'hands_female', 'hands_children', 'tot_wages', 'months_active',
       'materials_value', 'output', 'ind_detailed', 'ind_broadest',
       'ind_leontief', 'ind_granular', 'materials_value1', 'materials_value2',
       'materials_value3', 'materials_value4', 'materials_value5', 'output1',
       'output2', 'output3', 'output4', 'output5'],
      dtype='object')

In [64]:
def ratio(df, var1, var2):
    df[f'{var1}&{var2}'] = df[var1]/df[var2]

def ratio_tot_wages(df, var1, var2, var3, var4):
    df[f'{var1}&{var2}&{var3}&{var4}'] = (df[var1] + df[var2]+ df[var3])/df[var4]

In [65]:
#create specific ratios
ratio(input_1870, 'materials_value', 'output')
ratio(input_1870, 'materials_value', 'capital')
ratio(input_1870, 'capital', 'output')
ratio(input_1870, 'capital', 'materials_value')
ratio(input_1870, 'output', 'materials_value')
ratio(input_1870, 'output', 'capital')
ratio_tot_wages(input_1870, 'hands_male', 'hands_female', 'hands_children', 'tot_wages')
ratio(input_1870, 'tot_wages', 'output')

var_list = ["capital", "materials_value", "hands_female", "hands_male", "tot_wages", "output", 'materials_value&output', 
            'materials_value&capital', 'capital&output', 'capital&materials_value', 
            'output&materials_value', 'output&capital','hands_male&hands_female&hands_children&tot_wages', 'tot_wages&output',
            'months_active'
            ]

input_1870.replace({np.inf: 0, -np.inf: 0, np.nan: 0}, inplace=True)

In [66]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis

industry_list = input_1870["ind_broadest"].unique().tolist()
final_df = []

for industry in industry_list:
    input_df = input_1870[input_1870["ind_broadest"] == industry].copy()  # Ensure a copy

    if input_df.empty:
        continue  # Skip if there's no data for this industry

    # Compute mean and covariance matrix
    mean_vec = np.mean(input_df[var_list], axis=0)
    cov_matrix = np.cov(input_df[var_list], rowvar=False)

    # Check if covariance matrix is singular
    if np.linalg.det(cov_matrix) == 0:
        print(f"Skipping {industry}: Covariance matrix is singular")
        continue

    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Compute Mahalanobis distance for each firm
    distances = np.array([mahalanobis(row, mean_vec, inv_cov_matrix) for row in input_df[var_list].to_numpy()])
    
    # Compute contributions
    contributions = np.array([(row - mean_vec) * np.dot(inv_cov_matrix, (row - mean_vec)) for row in input_df[var_list].to_numpy()])
    contributions_df = pd.DataFrame(contributions, columns=[col + "_con" for col in var_list])

    # Identify the most anomalous variable for each row
    input_df['most_anomalous_var'] = contributions_df.idxmax(axis=1)
    input_df['mahalanobis'] = distances

    # Combine input_df with contributions_df (ensuring correct alignment)
    input_df = pd.concat([input_df.reset_index(drop=True), contributions_df.reset_index(drop=True)], axis=1)

    final_df.append(input_df)  # Append a COPY

# Combine all industry DataFrames into one
final_df = pd.concat(final_df, ignore_index=True)

In [67]:
final_df['dist_squared'] = final_df['mahalanobis'] ** 2
for var in var_list:
    final_df[var + "_divided"] = final_df[var+"_con"]/final_df['dist_squared']
final_df['total'] = 0
for var in var_list:
    final_df['total'] = final_df['total'] + final_df[var + "_divided"]
final_df

,file_name,firm_number,firm_name,capital,hands_male,hands_female,hands_children,tot_wages,months_active,materials_value,...,materials_value&output_divided,materials_value&capital_divided,capital&output_divided,capital&materials_value_divided,output&materials_value_divided,output&capital_divided,hands_male&hands_female&hands_children&tot_wages_divided,tot_wages&output_divided,months_active_divided,total
0,AL_7__00098_l_L.jpg,1,Jos Wilson,1000.0,2.0,0.0,1.0,150.0,4.0,4000.0,...,-0.008766,0.008091,0.266827,0.023580,0.000677,-0.000271,0.008597,0.304188,0.033397,1.0
1,AL_7__00165_r_L.jpg,1,Allison James,170.0,1.0,0.0,0.0,0.0,8.0,50.0,...,0.140005,0.071682,-0.013339,0.029260,0.002569,0.337114,0.004223,0.208118,0.017401,1.0
2,AL_7__00165_r_L.jpg,6,La? Brown,250.0,1.0,0.0,0.0,0.0,12.0,250.0,...,0.000923,0.031762,0.009267,0.004906,-0.000326,0.462271,0.004306,0.259527,0.019666,1.0
3,AL_7__00165_r_L.jpg,7,Scott Thomas,300.0,0.0,0.0,0.0,155.0,10.0,250.0,...,0.077758,0.067747,0.033177,0.001466,-0.000181,0.412193,0.005694,0.011563,0.025219,1.0
4,AL_7_autauga_00003_r_L.jpg,7,S S Booth & Co,3000.0,4.0,0.0,0.0,2000.0,12.0,3700.0,...,0.014874,0.041116,-0.008128,0.007979,-0.000353,0.609247,0.005448,0.118135,0.024152,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204273,dc_7_dist columbia_00211_l_L.jpg,10,Dunkhost Wm,1200.0,1.0,0.0,0.0,500.0,12.0,1000.0,...,0.003100,0.112575,0.002814,0.116297,-0.021145,0.154460,0.006809,0.031187,0.016631,1.0
204274,dc_7_dist columbia_00211_r_L.jpg,4,Gasch H,2000.0,1.0,0.0,0.0,600.0,12.0,850.0,...,0.071464,0.172247,0.009186,0.065046,0.017413,0.130298,0.005197,0.052744,0.015547,1.0
204275,dc_7_dist columbia_00211_r_L.jpg,10,Sellhausen F. W.,2000.0,1.0,0.0,0.0,500.0,12.0,600.0,...,0.045572,0.087126,0.085208,0.280174,0.029799,0.073182,0.002762,0.149026,0.008556,1.0
204276,dc_7_dist columbia_00212_r_L.jpg,1,Day Saml E,3000.0,3.0,0.0,0.0,1800.0,12.0,1460.0,...,0.023290,0.040821,-0.025184,0.013534,0.014682,0.052405,0.001927,0.833303,0.005258,1.0


In [68]:

for var in var_list:
    batch = []
    for ind in industry_list:
        ind_specific = final_df[final_df["ind_broadest"] == ind].copy()  # Ensure a copy
        #ORIGINAL CUTOFF : .0001
        quantile_low = ind_specific[var + '_divided'].quantile(0.0001)
        quantile_high = ind_specific[var + '_divided'].quantile(1 - 0.0001)

        # Filter the lowest and highest 0.01% values
        lowest_values = ind_specific[ind_specific[var + '_divided'] <= quantile_low]
        highest_values = ind_specific[ind_specific[var + '_divided'] >= quantile_high]

        # Combine results
        result = pd.concat([lowest_values, highest_values])

        batch.append(result)

    batch = pd.concat(batch, ignore_index=True)

    if "materials_value&output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "hands_male", "hands_female", "hands_children", "tot_wages", "months_active",
                           "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5",
                           "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)
    elif "output&materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "hands_male", "hands_female", "hands_children", "tot_wages", "months_active",
                    "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5",
                    "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)

    elif "materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "hands_male", "hands_female", "hands_children", "tot_wages", "months_active",
                    "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5",
                    "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "materials_value" in left:
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
        else:
            variable_check['correct_materials_value_1'] = None
            variable_check['correct_materials_value_2'] = None
            variable_check['correct_materials_value_3'] = None
            variable_check['correct_materials_value_4'] = None
            variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)

    elif "output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "hands_male", "hands_female", "hands_children", "tot_wages", "months_active",
                            "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5",
                            "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "output" in left:
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2']= None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2'] = None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
        else:
            variable_check['correct_output_value_1'] = None
            variable_check['correct_output_value_2'] = None
            variable_check['correct_output_value_3'] = None
            variable_check['correct_output_value_4'] = None
            variable_check['correct_output_value_5'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)

    else:
        parts = var.split("&")
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "hands_male", "hands_female", "hands_children", "tot_wages", "months_active",
                    "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5",
                    "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        if len(parts) == 2:
            left, right=parts
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{left}'] = None
            variable_check[f'correct_{right}'] = None
        else:
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{var}'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)


/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_3436/776381975.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variable_check['transcription_error'] =  None
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_3436/776381975.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variable_check[f'correct_{var}'] = None
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_3436/776381975.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram